In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
cols = ['ID(original)', 'ID' , 'Q' , 'A']
data = pd.read_csv('COQB-19_crowdsourced_06202020.csv', sep=',', names=cols, header=None, encoding='latin-1', skiprows=1)
data = data.dropna()
ID= data['ID'].tolist()
corpus = data['Q'].tolist()
Answers = data['A'].tolist()

In [ ]:
data.head()

,ID(original),ID,Q,A
0,5GTrue,1,is it true that 5G causes the Corona virus?,No. This is one of the myths that was busted b...
1,5GTrue,1,Can you get Covid 19 from 5g?,No. This is one of the myths that was busted b...
2,Affected,2,What are the different risk categories and wha...,"While all ages can be infected by the virus, t..."
3,Affected,2,Who tends to catch the virus?,"While all ages can be infected by the virus, t..."
4,Affected,2,What age brackets are most affected by the cor...,"While all ages can be infected by the virus, t..."


In [ ]:
pairs = list(zip(corpus,Answers))

In [ ]:
max_answer_words = 50

In [ ]:
input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs[:]:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc)[:max_answer_words])
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])


In [ ]:
max_decoder_seq_length

56

In [ ]:
max_encoder_seq_length

27

In [ ]:
batch_size = 256

In [ ]:
def encode_question_answer(input_doc, target_doc):
  """ Encode the question andanswer """
  encoder_input_data = np.zeros(
    (max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
  decoder_input_data = np.zeros(
     (max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
  decoder_target_data = np.zeros(
    (max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
  
  for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
    #Assign 1. for the current line, timestep, & word in encoder_input_data
    encoder_input_data[timestep, input_features_dict[token]] = 1
    
  for timestep, token in enumerate(target_doc.split()):
    decoder_input_data[timestep, target_features_dict[token]] = 1
    if timestep > 0:
      decoder_target_data[timestep - 1, target_features_dict[token]] = 1  
      
  return (encoder_input_data,  decoder_input_data),  (decoder_target_data,decoder_target_data)

In [ ]:
def generator_question_answer():
  """ Encode the question andanswer """
  idx = 0
  while idx < len(pairs):
    yield encode_question_answer(input_docs[idx], target_docs[idx])
    idx +=1

In [ ]:
dataset = tf.data.Dataset.from_generator(generator_question_answer,
                                          output_types=((tf.float32, tf.float32), (tf.float32, tf.float32)),
                                          output_shapes = (([max_encoder_seq_length, num_encoder_tokens],[max_decoder_seq_length,num_decoder_tokens]),
                                          ([max_decoder_seq_length,num_decoder_tokens],[max_decoder_seq_length,num_decoder_tokens])),)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
epochs = 600
#Encoder
encoder_inputs = Input(shape= (max_encoder_seq_length, num_encoder_tokens), dtype=tf.float32, name='enc_input')
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(max_decoder_seq_length, num_decoder_tokens), dtype=tf.float32, name='dec_input')
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
training_model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=[decoder_outputs,decoder_outputs])
#Compiling
training_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.003), loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit(dataset, epochs = epochs)
training_model.save('training_model.h5')

Epoch 1/600
10/10 [==============================] - 26s 249ms/step - loss: 10.9869 - dense_loss: 5.4934 - dense_1_loss: 5.4934 - dense_accuracy: 0.0428 - dense_1_accuracy: 0.0428
Epoch 2/600
10/10 [==============================] - 3s 262ms/step - loss: 9.3091 - dense_loss: 4.6545 - dense_1_loss: 4.6545 - dense_accuracy: 0.0429 - dense_1_accuracy: 0.0429
Epoch 3/600
10/10 [==============================] - 3s 266ms/step - loss: 9.1839 - dense_loss: 4.5919 - dense_1_loss: 4.5919 - dense_accuracy: 0.0409 - dense_1_accuracy: 0.0409
Epoch 4/600
10/10 [==============================] - 3s 266ms/step - loss: 9.1713 - dense_loss: 4.5857 - dense_1_loss: 4.5857 - dense_accuracy: 0.0394 - dense_1_accuracy: 0.0394
Epoch 5/600
10/10 [==============================] - 3s 258ms/step - loss: 9.0264 - dense_loss: 4.5132 - dense_1_loss: 4.5132 - dense_accuracy: 0.0437 - dense_1_accuracy: 0.0437
Epoch 6/600
10/10 [==============================] - 3s 265ms/step - loss: 9.0198 - dense_loss: 4.5099 - den

In [15]:
from keras.models import load_model
training_model = load_model('training_model.h5')

In [16]:

encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

In [17]:
latent_dim = 256
decoder_infer_inputs = Input(shape=(1, num_decoder_tokens), dtype=tf.float32, name='dec_infer_input')
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_infer_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_infer_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)


In [18]:
def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    word_idx = 0
    stop_condition = False
    while not stop_condition:
        #Predicting output tokens with probabilities and states
        output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
        sampled_token_index = np.argmax(output_tokens[0, 0, :])
        sampled_token = reverse_target_features_dict[sampled_token_index]
        decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
        if (sampled_token == '<END>' or word_idx >= max_decoder_seq_length-1):
          stop_condition = True
        else:
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          word_idx+=1
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [ ]:
class ChatBot:
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()